In [27]:
import os
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time
import catalog_utils as cu

In [2]:
# Read ZTF metadata from file. ZTF table file includes multiple header lines throughout the file.
colnames = ['UT_START', 'sun', 'exp', 'filter', 'obstype', 'field', 'pid', 'ra', 'dec', 'slew', 'wait', 
            'fileroot', 'programpi', 'queue_comment']
ztf_meta = pd.read_table('ztf_exposures.tbl', delim_whitespace=True, comment='|', names=colnames)
print('all metadata = %d exp' % len(ztf_meta))
ztf_meta = ztf_meta.query('queue_comment != "calib"')
print('non calibration exp = %d exp' % len(ztf_meta))

all metadata = 72286 exp
non calibration exp = 63678 exp


In [66]:
# Pull out the times - must convert to str from 'object' format Pandas read.
times = np.array(ztf_meta.UT_START.values, 'str')
# Convert times to MJD units.
t = Time(np.array(ztf_meta.UT_START.values, 'str'), format='isot', scale='utc')
ztf_meta['mjd'] = t.mjd

# And cut out images which predate our alert stream
t_start = Time('2018-06-18T23:30:00', format='isot', scale='utc')
ztf_meta = ztf_meta.query('mjd >= @t_start.mjd')
print('exposures after time cut = %d' % len(ztf_meta))

exposures after time cut = 15907


In [67]:
cols = ['UT_START', 'mjd', 'ra', 'dec', 'filter', 'exp']
ztf_meta[cols][0:5]

,UT_START,mjd,ra,dec,filter,exp
54119,2018-06-19T04:05:26.7,58288.170448,216.9938,-9.85,3,30
54120,2018-06-19T04:06:06.0,58288.170903,220.6100,-2.65,3,30
54121,2018-06-19T04:06:45.3,58288.171358,219.6171,4.55,3,30
54122,2018-06-19T04:07:25.0,58288.171817,215.1253,11.75,3,30
54123,2018-06-19T04:08:04.4,58288.172273,216.2361,18.95,3,30


In [68]:
# Let's read some orbits
catdir = 'catalogs'
mpcfile = 'tmp_mpc.dat'
astfile = 'tmp_ast.dat'

In [69]:
mpc = cu.read_mpcorb(os.path.join(catdir, mpcfile))

In [70]:
ast = cu.read_astorb(os.path.join(catdir, astfile))

In [71]:
from lsst.sims.movingObjects import Orbits, PyOrbEphemerides
from lsst.sims.utils import angularSeparation
from lsst.sims.movingObjects import DirectObs

In [94]:
obj = mpc.iloc[100]

In [95]:
orb = Orbits()
orb.setOrbits(deepcopy(vesta))

In [96]:
orb.orbits

,mpcId,H,G,epoch,meanAnomaly,argPeri,Omega,inc,e,meanDailyMotion,...,precisePerts,computer,readableName,lastObs,numberId,Name,Desig,objId,g,sed_filename
100,101,8.33,0.35,51991.0,18.38457,348.3317,343.40242,10.19517,0.14,0.237351,...,38h,MPCLINUX,(101) Helena,20171216,101,Helena,NULL,101,0.15,S.dat


In [97]:
oorbeph = PyOrbEphemerides()
oorbeph.setOrbits(orb)

In [98]:
# %%timeit
times = ztf_meta.mjd.values
eph = oorbeph.generateEphemerides(times, obscode='I41')
ephs = eph[0]

In [100]:
ephs['ra'], ephs['dec']

(array([ 190.86397311,  190.86402702,  190.86408094, ...,  195.86359187,
         195.86370357,  195.8638125 ]),
 array([ 2.77643525,  2.77638841,  2.77634157, ..., -0.55335446,
        -0.5534184 , -0.55348074]))

In [101]:
separation_from_ztf_fields = angularSeparation(ephs['ra'], ephs['dec'], 
                                               ztf_meta['ra'].values, ztf_meta['dec'].values)

In [102]:
ztf_rfov = np.sqrt(47.)/np.pi
idxs = np.where(separation_from_ztf_fields < ztf_rfov)

In [103]:
ephs['magV'][idxs]

array([], dtype=float64)

In [64]:
Time(ephs['time'][idxs], format='mjd').isot

array(['2018-03-27T03:33:20.400', '2018-03-27T04:41:51.800',
       '2018-03-27T05:48:14.500', '2018-03-28T05:47:45.100',
       '2018-03-29T04:36:25.200', '2018-03-30T04:49:08.600',
       '2018-03-30T05:25:56.600', '2018-03-30T05:48:04.400',
       '2018-03-31T05:21:50.100', '2018-04-02T03:09:55.000',
       '2018-04-02T03:22:23.900', '2018-04-02T04:37:39.400',
       '2018-04-06T03:16:01.900', '2018-04-06T03:55:59.500',
       '2018-04-07T04:15:00.300', '2018-04-23T03:27:05.000',
       '2018-04-23T04:40:51.700', '2018-04-26T03:57:19.600',
       '2018-04-26T05:04:23.600', '2018-04-29T04:34:35.500',
       '2018-05-08T03:48:05.000', '2018-05-08T04:13:20.400',
       '2018-05-11T03:43:22.100', '2018-05-11T04:28:51.500'],
      dtype='<U23')

In [56]:
len(idxs[0])

24

In [104]:
# Did this show up in alerts?
import mysql.connector as mariadb
alldf = pd.read_sql_query('select * from alerts where ssdistnr > -1 and jd > %f' % (t_start.jd), con)

ModuleNotFoundError: No module named 'mysql'